# L8 - Replenishment about Prompt Engineering

## 1. 什么是Prompt? --- 从人机交互（HCI）视角出发

上世纪80年代至今，从命令行（CLI）到图形界面（GUI）再到自然用户界面（NUI），从最开始的命令行输入演变为现在通过语音、手势、眼球等多种自然动作进行交互，人机交互的形态正在不断更新迭代。

换个角度想，如果将大语言模型（LLM）视作一种特殊的系统或计算设备，此时Prompt在目的上与手势、声音以及眼球等并无太大区别————都是为了与设备进行交互，来以此操作设备获取想要的结果。因此在某种程度上来看，Prompt实际上也可被理解为一种全新的自然用户界面。即通过Prompt，用户可以直观的告诉LLM想要它做什么，来以此控制模型去做什么，并获得想要的结果。

![](img/Prompt_NUI.png)

### Prompt的本质

![](img/Prompt_structure.png)

对于LLM而言，大部分的Prompt都是text文本，由Instruction（指令）以及Content（内容）两部分构成，其中【指令】部分是我们想要LLM去执行的命令，【内容】部分则是提供给LLM的知识或材料，例如：

In [1]:
import openai
import os

os.environ["all_proxy"] = "http://127.0.0.1:<端口号>"
openai.api_key  = 'sk-'

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]

In [3]:
text = """今天天气糟透了！"""
prompt = f"""
你的任务是根据内容判断句子情感\
内容：```{text}```
"""

response = get_completion(prompt)
print(response)

负面情感。


在上述内容中：
- Instruction：你的任务是根据内容判断句子情感
- Content：今天天气糟透了！

二者共同构成了一个完整的Prompt

但是Prompt在构建时并非严格遵守这种【Prompt=Instruction + Content】的格式，一些较为简短的Prompt，例如“今中国的首都在哪里？”其中仅包括指令，省去了内容。此时可以认为Content部分为空，上述结构依旧适用。

In [5]:
prompt = """
中国的首都在哪里？
"""
response = get_completion(prompt)
print(response)

中国的首都是北京。


此外，Prompt的形式也并非仅局限于【文本】，在视觉任务中，也可采用Bounding box、Point等视觉要素作为视觉Prompt，例如SAM（Segment Anything Model），也可在某些情况下采用【语音】作为Prompt，例如邱锡鹏团队所提出的SpeechGPT等。

Teven等人通过大量实验发现，引入一条Prompt相当于多标注了100组真实数据样本。因此Prompt在下游任务面对数据缺乏的场景或者Zero-shot场景时，有着无可比拟的优势，无需微调大模型，仅靠Prompt Tuning的方式即可显著提升性能。

### Prompt 类型

目前Prompt的形式主要分为以下几种：

**1. Zero-shot Prompt：** 不提供参考示例，直接通过指令让大模型完成任务，也就是上述提到的【Content】为空的情况。

In [6]:
prompt = """用莎士比亚的风格写一首诗"""
response = get_completion(prompt)
print(response)

Oh, fair maiden, with eyes so bright,
Thou art a vision of pure delight.
Thy beauty doth rival the sun's warm glow,
And in thy presence, my heart doth overflow.

Thou art a rose, with petals so fine,
And in thy grace, all virtues combine.
Thy voice is music, sweet to the ear,
And in thy company, all troubles disappear.

Oh, how I long to hold thee near,
And whisper sweet nothings in thy ear.
Thou art the light that guides my way,
And in thy love, I shall forever stay.

So let us dance, beneath the moon's soft light,
And revel in the magic of this night.
For in thy arms, I find my home,
And in thy love, I shall never roam.


**2. Few-shot Prompt：**【Content】部分不为空，给出少量示例供大模型在完成任务时参考。

In [8]:
text = """
戍鼓断人行，秋边一雁声。
露从今夜白，月是故乡明。
有弟皆分散，无家问死生。
寄书长不达，况乃未休兵！
"""

prompt = f"""
模仿下面反引号中句子的风格，写一首七言律诗
内容：```{text}```
"""

response = get_completion(prompt)
print(response)

戍鼓声断秋边行，一声雁鸣入耳听。
露珠今夜满衣裳，故乡明月照人情。
弟兄分散无依靠，死生问路何时平。
寄书未达心不宁，未休兵戈何时停。


但是Few-shot也有一些缺点需要注意：
1. 多标签偏见：在提供给LLM的案例中，如果存在标签分布不平衡的情况，会导致对推理结果造成负面影响。
2. 临近偏差：模型倾向于为测试样本输出最近看到的few-shot样本。这个听上去有点像灾难遗忘，即模型更喜欢记住最近输入的样本信息。
3. 共同标记偏差：输入Few-shot的内容中存在大量重复token时，LLM的输出结果中也会更容易生成这些token，因为LLM的本质是一个概率模型，重复token越多，对应概率越大。

设计Few-shot Prompt的一些技巧：
1. 尽量使用与target部分相似的案例作为参考，可采用KNN进行筛选。
2. 对输入案例进行重新排序，来降低多标签偏见以及临近偏差的影响。
3. 使用强化学习或主动学习的方法选择案例（过于繁琐）

**3. Role Prompt：** 让大模型扮演一个角色，例如猫娘，来获得更好的交互效果。

In [9]:
prompt = f"""
现在你需要扮演一只猫娘，请以猫娘的语气和我对话。
"""

response = get_completion(prompt)
print(response)

喵~ 你好啊，我是一只可爱的猫娘，有什么可以帮到你的吗？喵~


**4.Instruction prompt：** 指令形式的prompt，个人感觉与Zero-shot并无太大区别。

In [13]:
prompt = """
介绍一下银河系
"""
response = get_completion(prompt)
print(response)

银河系是地球所在的星系，也是我们所知道的最大的星系之一。它是由数百亿颗恒星、行星、气体和尘埃组成的巨大星云，直径约10万光年，厚度约1000光年。银河系的中心有一个超大质量黑洞，质量约为4百万个太阳质量。银河系中的恒星和行星都围绕着中心旋转，形成了一个旋转的盘状结构。银河系中的恒星和行星分布不均，有些区域比其他区域更密集，这些区域被称为星云。银河系是我们了解宇宙的重要窗口，也是我们探索宇宙的基础。


**5. Chain-of-thought Prompt：** 思维链（CoT），令LLM将多步推理问题分解为中间步骤，以此将额外的计算分配给需要推理的问题，通过这种让大模型一步步思考的方式来求解较难的推理问题。（在逻辑推理任务中效果不错）

In [16]:
prompt = """
我去市场买了10个苹果。我给了邻居2个苹果和修理工2个苹果。然后我去买了5个苹果并吃了1个。我还剩下多少苹果？
"""
response = get_completion(prompt)
print(response)

你现在手上还有12个苹果。


In [17]:
prompt = """
我去市场买了10个苹果。我给了邻居2个苹果和修理工2个苹果。然后我去买了5个苹果并吃了1个。我还剩下多少苹果？
请逐步思考并给出步骤
"""
response = get_completion(prompt)
print(response)

1. 我买了10个苹果。
2. 我给了邻居2个苹果，还剩下8个苹果。
3. 我给了修理工2个苹果，还剩下6个苹果。
4. 我又买了5个苹果，一共有11个苹果。
5. 我吃了1个苹果，还剩下10个苹果。 

所以，我还剩下10个苹果。


CoT的使用技巧：
1. 尽量令temperature > 1，以此生成多样化的结果进行融合
2. 在提供给LLM一些few-shot案例时，尽量进行随机化处理，来增加案例多样性

**6. Multimodel Prompt：** 多模态Prompt，输入的Prompt不再是单一的文本信息，而是包含了诸多模态，通过多种模态来与LLM进行交互。（由于GPT-4 API 尚未开放，故不做演示）

## 2. 如何高效利用Prompt完成任务需求？

### 1. 关键假设

在使用LLM之前，我们需要给自己一个假设，即大模型不会犯错，永远是 **正确的**，之所以输出错误的结果，是我们的Prompt写的有问题，
但实际上这种假设并不符合常理，之所以这样，是为了通过这种方式来使得我们将精力聚焦于Prompt的设计和调教，而非大模型的正确与否上。

### 2. LLM的使用规则

- 规则1：相较于Zero-shot，Few-shot为LLM提供了参考案例（In-context learning），往往会有着更好的效果。
- 规则2：可以通过赋予角色的方式，来提高LLM在特定任务上的性能。当然，这种方法并不是总是有效（实际上在大部分任务中并没有明显效果）
- 规则3：在推理任务中尽量使用CoT，来诱导模型一步一步进行思考
- 规则4：为了避免Prompt与Context中的内容混淆，可以采用一些符号来分隔Prompt与Context中的内容。
- 规则5：善用温度变量（temperature），在ChatGPT API中，温度变量决定了模型输出结果的随机性
    - temperature=0：模型输出结果是确定的，通常用于决定性任务，如分类、预测等；
    - temperature>0：模型随机性越强，结果更具多样性，适合生成任务。

## 3. 自动学习最优Prompt（Prompt Tuning）

除了手工调整Prompt外，机器同样也可自动生成Prompt，这种做法就是Prompt Tuning。

![](img/Prompt_Tuning.png)

刘鹏飞等人在[Pre-train, Prompt, and Predict](https://arxiv.org/abs/2107.13586)这篇论文中将Prompt Tuning拆分为了5个方面：
1. Promptless fine-tuning: 不使用prompt、直接微调语言模型。
2. Tuning-free prompting: 无需微调的prompt。
3. Fixed-LM prompt tuning: 固定LM，微调prompt。这种也是最为常用的微调方法。
4. Fixed-prompt LM tuning: 固定prompt，微调LM。
5. Prompt+LM fine-tuning: Prompt和LM两部分一起微调。

![](img/Prompt_Tuning_2.png)

### 1. Tuning-free Prompt

代表性的工作有AutoPrompt，令计算机通过梯度优化的方法，自动从一系列候选词中生成对应目标任务的Prompt，以实现Prompt的自动调节
![](img/AutoPrompt.png)

### 2.Fixed-LM Prompt Tuning

代表性工作为Prefix-Tuning，将LLM的微调由传统的微调全部Transformer层改为在Transformer的输入层中加入可学习的Prompt Embedding向量层，通过对Embedding层微调，实现对LLM的微调，在保证效果的同时，降低了对硬件资源的依赖。

![](img/Prefix-Tuning.png)

在Prefix-Tuning中，Prefix实际上可以看成对模型的每一层增加可以调节的参数。具体表现为对Transformer结构中注意力内的键值对（Key-Value）进行扩展

Prefix-Tuning技术最早应用在自然语言生成任务中，由$\text{[Prefix,x,y]}$三部分组成，其中Prefix为前缀，x为输入，y为输出。Prefix-tuning将预训练参数固定，Prefix参数进行微调：不仅只在embedding上进行微调，也在TransFormer上的embedding输入每一层进行微调。

另一种Prompt微调方式为P-Tuning，为了消除手工设计Prompt所带来的不稳定现象，研究人员将自然语言提示的token换为可训练的嵌入，利用LSTM进行重新参数化来加速训练，并引入少量自然语言提示的锚点字符（蓝色），以此提升效果。

![](img/p-tuning_v1.png)

上图中可以看到，对于给定的Prompt T=$\{[P_{0:i},x,[P_{i+1:m}]\}$，传统的离散搜索会将其映射为嵌入表示$\{e([P_{0:i}]),e(x),e([P_{i+1,m}]),e(y)\}$，其中$[P_i]$所用的词语是模型词汇表中的词。
而P-Tuning的方式除了引入嵌入表示$e$之外，还会将$[P_i]$当初一个可学习的token $h$，该token会伴随着微调不断更新，最后映射为$\{h_0,...,h_i,e(x),h_{i+1},...,h_m,e(y)\}$，这些可学习的token可以与嵌入表示一起用于生成过程，从而提供更加定制和有针对性的信息。

在整个P-Tuning流程中，给定输入句子以及一个预设计的离散模板，在模板中选择一个或多个token作为伪标记来表示任务的提示部分，引导生成过程。这些伪标记并非随机初始化后直接训练，而是会输入到一个可学习的BiLSTM以及使用ReLU作为激活函数的双层MLP中，计算出Embedding映射为隐状态向量作用于生成过程。

但研究人员发现，P-Tuning v1虽然显著提高了生成效果，其Anchor字符仍需进行手工设计，而非完全自动化的流程。并且只有模型参数到达10B量级，P-Tuning的效果才能比肩Fine-Tuning，在小模型上的效果表现并不好。

![](img/p-tuning_v2_1.png)

为了提高P-Tuning在小模型上的效果，P-Tuning v2引入了Prefix-Tuning技术，P-Tuning v2本质是将Prefix-Tuning用在了自然语言理解任务中，作者发现P-Tuning中采用BiLSTM表征伪标记的做法是导致推理能力不足的主要原因，为此使用Prefix-Tuning中的深层模型替换BiLSTM，不仅仅在输入层，而是在每一层都添加一部分可训练参数，利用多层提示（深度提示优化）的方法，对每层Transformer的embedding输入进行微调。

![](img/p-tuning_v2_2.1.png)
![](img/p-tuning_v2_2.2.png)

通过这种做法，不同层中的提示作为前缀token加入到输入序列中，并独立于其他层间（而不是由之前的transformer层计算）。一方面，P-tuning v2有更多的可优化的特定任务参数（从0.01%到0.1%-3%），以允许更多的每个任务容量，而它仍然比完整的预训练语言模型小得多；另一方面，添加到更深层的提示，可以对输出预测产生更直接和重大的影响。